#### Using pretrained CNN's is possible with transfer learning

In this notebook, I will load the pretrained model ResNet from Keras applications and used the pretrained model weights to investigate whether these models can be transfered from one set of data to another. The model was originally trained on ImageNet data, with image sizes of 224x224. This type of network won first place on the ILSRVC 2015 image classification task, although that model was significantly deeper than the 50 layer version imported here. (152 layers for the winning model.)

The original paper can be found here: https://arxiv.org/abs/1512.03385

In [1]:
#import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
%run __initremote__.py

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_acc', 
                                           min_delta=0, 
                                           patience=5, 
                                           verbose=0, 
                                           mode='auto')

In [12]:
resnet_base = keras.applications.resnet50.ResNet50(include_top=False, 
                                     weights='imagenet', pooling = 'avg')

In [13]:
resnet_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        input_2[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_50 (Activation)      (None, None, None, 6 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [14]:
x_train.shape

(50000, 32, 32, 3)

In [16]:
x_train[0].shape

(32, 32, 3)

The CIFAR-10 images come in size 32x32, but the ResNet50 model was trained on the ImageNet images, which are 225x225. In order for the model to run correctly, the input shape must be the same. This requires resizing the images prior to predicting on them with the pretrained weights. 

In [22]:
import numpy as np
import scipy

In [23]:
big_x_train = np.array([scipy.misc.imresize(x_train[i], (225, 225, 3)) 
                            for i in range(0, len(x_train))]).astype('float32')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


In [24]:
big_x_train.shape

(50000, 225, 225, 3)

In [25]:
x_preds_res_train = resnet_base.predict(big_x_train)

In [27]:
x_preds_res_train.shape

(50000, 2048)

In [28]:
big_x_test = np.array([scipy.misc.imresize(x_test[i], (225, 225, 3)) 
                            for i in range(0, len(x_test))]).astype('float32')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


In [ ]:
x_preds_res_test = resnet_base.predict(big_x_test)

In [32]:
x_preds_res_test

array([[  1.58830166e-01,   2.10372090e+00,   7.57917613e-02, ...,
          0.00000000e+00,   6.19659126e-02,   2.05141321e-01],
       [  0.00000000e+00,   8.44799280e-02,   3.16568851e-01, ...,
          9.98439118e-02,   2.28154752e-02,   1.93401456e+00],
       [  0.00000000e+00,   8.04948583e-02,   5.90594427e-04, ...,
          1.04155672e+00,   4.05220568e-01,   0.00000000e+00],
       ..., 
       [  2.21008677e-02,   1.50756168e+00,   1.75811313e-02, ...,
          4.49203998e-01,   1.33018298e-02,   0.00000000e+00],
       [  3.49001922e-02,   1.91300020e-01,   6.66999351e-03, ...,
          4.92483169e-01,   3.58047374e-02,   6.81567848e-01],
       [  3.54255259e-01,   1.20027447e+00,   1.33867726e-01, ...,
          1.89002335e+00,   3.25860560e-01,   1.37407975e-02]], dtype=float32)

In [33]:
x_preds_res_test.shape

(10000, 2048)

In [34]:
import pickle
pickle.dump(x_preds_res_train, open("x_preds_res_train", "wb"))
pickle.dump(x_preds_res_test, open("x_preds_res_test", "wb"))

In [36]:
x_preds_res_train.shape[1:]

(2048,)

In [55]:
output = Sequential()
output.add(Dense(512, input_shape=x_preds_res_train.shape[1:]))
output.add(Activation('relu'))
output.add(Dense(10))
output.add(Activation('softmax'))

In [56]:
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-5)

output.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

history_resnet = output.fit(x_preds_res_train, y_train,
              batch_size=32,
              epochs=100,
              validation_data=(x_preds_res_test, y_test),
              shuffle=True,
              callbacks=[early_stop])

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 8s 164us/step - loss: 0.5782 - acc: 0.8040 - val_loss: 0.4636 - val_acc: 0.8400
Epoch 2/100
50000/50000 [==============================] - 7s 150us/step - loss: 0.3999 - acc: 0.8621 - val_loss: 0.4501 - val_acc: 0.8449
Epoch 3/100
50000/50000 [==============================] - 7s 150us/step - loss: 0.3448 - acc: 0.8817 - val_loss: 0.4036 - val_acc: 0.8595
Epoch 4/100
50000/50000 [==============================] - 7s 150us/step - loss: 0.3068 - acc: 0.8948 - val_loss: 0.3825 - val_acc: 0.8663
Epoch 5/100
50000/50000 [==============================] - 7s 150us/step - loss: 0.2759 - acc: 0.9048 - val_loss: 0.3780 - val_acc: 0.8668
Epoch 6/100
50000/50000 [==============================] - 7s 150us/step - loss: 0.2500 - acc: 0.9163 - val_loss: 0.3605 - val_acc: 0.8767
Epoch 7/100
50000/50000 [==============================] - 7s 149us/step - loss: 0.2264 - acc: 0.9234 - val_loss: 0.

Clearly, transfer learning is a quick, powerful way to obtain significant results. Above, the model reaches 88% accuracy after less than 20 epochs, already significantly better performance then previous models trained from 100 or more epochs. 

For production environments, it would be wise to investigate whether such pretrained models might be a powerful solution to specific learning tasks. The model above required significant investments of time, computing power, and research to obtain results that surpass human ability to classify images correctly. With such models available and easily transferable to other, similiar tasks, it is obvious they should be a go to solution under the right circumstances. 